In [1]:
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'

import multiprocessing as mp
mp.set_start_method("spawn", force=True)

import torch
torch.set_float32_matmul_precision('medium')
import numpy as np
import pytorch_lightning as pl
from tqdm import tqdm
import plotfancy as pf
import matplotlib.pyplot as plt
pf.housestyle_rcparams()

import sys
sys.path.append('../../mist-base/GW')
sys.path.append('../../mist-base/')
sys.path.append('../../mist-base/utils')
sys.path.append('../../')

from src.utils.generators import Simulator_Additive
from simulators.utils import *
from utils.data import OnTheFlyDataModule
from utils.module import CustomLossModule_withBounds

mycolors = ['#570f6d', "#9e8f92", '#f98e08']
folly = '#ff004f'

In [9]:
glob_sigma = 1
glob_bkg = True
glob_pve_bounds = False
glob_det = 'det'
glob_locks = [True, True, True] # mu amp sigma

# Nsims = 100_000
Nbins = 1024
train_bounds = 5

# simulator = Simulator_Additive(100,1,5,0.02,mode='gw', bkg=False)
simulator = Simulator_Additive(Nbins=Nbins, sigma=glob_sigma, bounds=train_bounds, 
                               fraction=0.2, bkg=glob_bkg, dtype=torch.float64, 
                               mode='gw', bump=glob_det, frange=[20,240],
                               lock_mu=glob_locks[0],lock_amp=glob_locks[1], lock_sigma=glob_locks[2])
simulator._init_gw()

Loading data for H1 at GPS time 1126259462.4
Loading PSD for H1 at GPS time 1126259462.4 [can take O(1 min)]
Loading posterior samples from ../../mist-base/GW/GW150814_posterior_samples.npz


In [10]:

# N_mc = 2e6

# batch_size = 2048*2
# N_batch = int(N_mc / batch_size)
# data_bin_H0 = []

# for _ in tqdm(range(N_batch)):
#     dummy = np.ones([124,len(simulator.grid)])*simulator.grid
#     mc_samples = simulator._fd_noise(124)
#     data_bin_H0.append(mc_samples)
    
# data_bin_H0 = np.concatenate(data_bin_H0)


# ### Start by ascertaining the data distribution ###
# from scipy.stats import norm
# def get_quantiles(indata, nsig:int, sigma_key=False):
#     data = np.sort(indata)
#     sigmas = np.arange(-nsig,nsig+1,1)
#     indices = (len(data)*norm.cdf(sigmas)).astype(int)
#     if not sigma_key:
#         return data[indices]
#     else:
#         return np.array([data[indices],sigmas])
    
# alldata = data_bin_H0.flatten()
# mean = np.mean(alldata)
# quantiles = get_quantiles(alldata, 3)
# quantiles_long = get_quantiles(alldata,5)

quantiles = np.array([0.05199686, 0.2145543,  0.58780088, 1.17737921, 1.91882754,
 2.75067576, 3.63463655])

quantiles_long = np.array([7.11978022e-04, 7.96148769e-03, 5.19968566e-02, 2.14554300e-01,
 5.87800876e-01, 1.17737921e+00, 1.91882754e+00, 2.75067576e+00,
 3.63463655e+00, 4.55164698e+00, 5.49045819e+00])

In [11]:
# dummy = np.ones([124,len(simulator.grid)])*simulator.grid
# a = simulator._fd_noise(124)
# pb = simulator._fd_theta_batched(124)
# b = simulator._fd_waveform_batched(pb)

# fig,ax = pf.create_plot()
# ax2 = fig.add_axes((1,0,0.3,1))
# plt.setp(ax2.get_yticklabels(), visible=False)

# ax.plot(simulator.grid,a[0], color='#ff004f', lw=0.5, alpha=1)

# ax.set_ylabel(r'$\tilde{d}_w(f)$')
# ax.set_xlabel(r'$f$')
# ax.set_xlim(1, 1024)

# ax2.hist(a[0], bins=50, orientation='horizontal', density=True, color='#ff004f')
# ax2.hist(a[0], bins=50, orientation='horizontal', density=True, color='black', histtype='step')
# ax2.set_xticks([0.25,0.50])
# ax2.set_xlim([0,0.69])

# for i,q in enumerate(quantiles_long):
#     extra = np.linspace(-100,2148, 100)
#     ax.fill_between(extra, quantiles_long[i], quantiles_long[-i],  color='#b0b0b0', alpha=0.15, zorder=-10)
#     ax2.fill_between(extra, quantiles_long[i], quantiles_long[-i],  color='#b0b0b0', alpha=0.15, zorder=-10)

# ax.plot(np.linspace(-100,2148, 100), quantiles[3]*np.ones(100), lw=3, c="#7f7f7f", zorder=-10)
# ax2.plot(np.linspace(-100,2148, 100), quantiles[3]*np.ones(100), lw=3, c="#7f7f7f", zorder=-10)


# pf.fix_plot([ax,ax2])

In [12]:
# dummy = np.ones([124,len(simulator.grid)])*simulator.grid
# a = simulator._fd_noise(124)
# pb = simulator._fd_theta_batched(124)
# b = simulator._fd_waveform_batched(pb)

# fig,ax = pf.create_plot()
# ax2 = fig.add_axes((1,0,0.3,1), sharey=ax)
# plt.setp(ax2.get_yticklabels(), visible=False)

# ax.plot(simulator.grid,a[0]+b[0], color='#ff004f', lw=0.5, alpha=1)

# ax.set_ylabel(r'$\tilde{d}_w(f)$')
# ax.set_xlabel(r'$f$')
# ax.set_xlim(20, 1024)
# ax.set_ylim(0,6)

# ax2.hist(a[0], bins=50, orientation='horizontal', density=True, color='#ff004f')
# ax2.hist(a[0], bins=50, orientation='horizontal', density=True, color='black', histtype='step')
# ax2.set_xticks([0.25,0.50])
# ax2.set_xlim([0,0.69])

# for i,q in enumerate(quantiles_long):
#     extra = np.linspace(-100,2148, 100)
#     ax.fill_between(simulator.grid, quantiles_long[i]+b[0], quantiles_long[-i]+b[0],  color='#b0b0b0', alpha=0.15, zorder=-10)
#     ax2.fill_between(extra, quantiles_long[i], quantiles_long[-i],  color='#b0b0b0', alpha=0.15, zorder=-10)

# ax.plot(simulator.grid, quantiles[3]+b[0], lw=3, c="#7f7f7f", zorder=-10)
# ax2.plot(np.linspace(-100,2148, 100), quantiles[3]*np.ones(100), lw=3, c="#7f7f7f", zorder=-10)


# pf.fix_plot([ax,ax2])

In [13]:
# # samples = simulator.sample(124)  
# obs = simulator.sample(1)
# # simulator.get_theta(100)

# test = simulator.sample(1)

# pf.housestyle_rcparams()
# fig, ax1 = pf.create_plot()

# plt.setp(ax1.get_xticklabels(), visible=False)
# ax2 = fig.add_axes((0,-.3,1,0.3), sharex=ax1)
# ax3 = fig.add_axes((1,-.3,0.2,0.3), sharey=ax2)
# plt.setp(ax3.get_xticklabels(), visible=False)
# plt.setp(ax3.get_yticklabels(), visible=False)
# ax4 = fig.add_axes((0,1,1,0.3), sharex=ax1)
# plt.setp(ax4.get_xticklabels(), visible=False)

# ax1.plot(simulator.grid_chopped,test['xi'][0], label=r'$x_i$', color="#d931f3", lw=0.5)
# ax1.plot(simulator.grid_chopped,test['x0'][0], label=r'$x_0$', color='#ff004f', lw=0.5)
# ax1.plot(simulator.grid_chopped,test['mu'][0], label=r'$\mu$', color='black')
# ax1.set_ylabel(r'$|\tilde{d}(f)|$')
# ax1.set_ylim([-3,8])
# ax1.legend(loc='upper right')
# ax1.set_xlim(20, 240)


# resd = test['x0'][0]-test['mu'][0]
# ax2.plot(simulator.grid_chopped,resd, color='#ff004f', lw=0.5)
# ax2.set_xlabel(r'$f$')
# ax2.set_ylabel(r'res ($x_0$)')
# ax2.set_ylim([0,4.4])
# for i in range(1,6):
#     ax1.fill_between(simulator.grid_chopped, quantiles_long[i]+test['mu'][0], quantiles_long[-i]+test['mu'][0],  color='#b0b0b0', alpha=0.15)
#     ax2.fill_between(simulator.grid_chopped, quantiles_long[i], quantiles_long[-i],  color='#b0b0b0', alpha=0.15)
#     ax3.fill_between(simulator.grid, quantiles_long[i], quantiles_long[-i],  color='#b0b0b0', alpha=0.15)

# ax3.hist(resd, orientation='horizontal', bins=14, edgecolor='black', color='#ff004f', density=True)
# ax3.set_xlim([0,1])

# for i in range(100):
#     ax4.plot(simulator.grid_chopped,simulator.sample(1)['mu'][0], lw=0.5, color='black', alpha=0.05)

# ax4.set_yticks([])
# ax4.set_ylim([0,2])
# pf.fix_plot([ax1,ax2, ax3,ax4])
# plt.tight_layout()

# plt.tight_layout()


In [14]:
from models.online_norm import OnlineStandardizingLayer
from models.resnet_1d import ResidualNet

class Network_epsilon(torch.nn.Module):
    def __init__(self, nbins):
        super().__init__()
        
        self.nbins = nbins

        self.logvariance = torch.nn.Parameter(torch.ones(self.nbins)*5)

        self.net = ResidualNet(1, 1, hidden_features=128, num_blocks=2, kernel_size=1, padding=0) 
        self.mu_predictor = torch.nn.Sequential(
            torch.nn.Linear(self.nbins, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, self.nbins)
        )

    def mu(self, x):
        x = self.mu_predictor(x.unsqueeze(1)).squeeze(1)
        return x
                
    def epsilon(self, x):
        resd = x - self.mu(x)
        out = self.net(resd.unsqueeze(1)).squeeze(1) # x-net
        return out
    
    def snr(self, x):
        return self.epsilon(x) / self.logvariance.exp().sqrt()  # [B, N_bins]
    
    def bounds(self):
        return self.logvariance.detach().exp().sqrt().mean(-1) * 5

        
    def forward(self, x):
        
        x0_block = x['x0']
        mu_block = x['mu']
        ni = x['ni']
        
        ###########################################
        epsilon_sim =  (2 * self.bounds() * torch.rand(x['x'].shape, 
                                                           device= x['x'].device, 
                                                           dtype= x['x'].dtype) - self.bounds()) * ni
        ###########################################

        data = x0_block+epsilon_sim
        
        # net evaluation_m
        net_mu = self.mu(data)
        error_mu = (net_mu-mu_block)**2
        l_mu = error_mu / (self.logvariance.exp() + 1e-10) + self.logvariance
        l_mu_return = l_mu.sum() * 0.5

        # net evaluation_e
        net_epsilon = self.epsilon(data)
        mask = ( ni != 0 )  
        squared_error_e = (net_epsilon - epsilon_sim)**2                                         # [B, N_bins]
        l_e = squared_error_e / (self.logvariance.exp() + 1e-21) + self.logvariance                   # [B, N_bins]
        l_e_return = (l_e * mask.float()).sum() * 0.5
        
        # combine
        return l_mu_return+l_e_return

    

In [ ]:
batch_size = 128

dm = OnTheFlyDataModule(simulator, Nsims_per_epoch=400*batch_size, batch_size=batch_size, num_workers=0)

network_epsilon = Network_epsilon(nbins=len(simulator.grid_chopped))
model = CustomLossModule_withBounds(network_epsilon, learning_rate=3e-3)
trainer = pl.Trainer(
    accelerator="gpu", 
    max_epochs=100, 
    precision=64,
    # fast_dev_run=True
)
trainer.fit(model, dm)
network_epsilon.cuda().eval();


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/data/tgh35/summer25/cam_venv/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:68: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type            | Params | Mode 
--------------------------------------------------
0 | model | Network_epsilon | 310 K  | train
--------------------------------------------------
310 K     Trainable params
0         Non-trainable params
310 K     Total params
1.240     Total estimated model params size (MB)
21        Modules in train mode
0         Modules in eval mode
/data/tgh35/summer25/cam_venv/lib/python3.11/site-pa

Training: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

: 

In [ ]:
torch.save(network_epsilon, f'networks/network_GW_q_1024_complex')
torch.save(model, f'networks/model_GW_q_1024_complex')
netid = 'GW_q_1024'

In [ ]:
# Convert tensors to scalars if they are tensors
train_loss_history = [loss.item() if hasattr(loss, 'item') else loss for loss in model.train_loss_history]
bounds_history = [bound.item() if hasattr(bound, 'item') else bound for bound in model.bounds_history]

# Generate a list of epoch numbers
epochs = range(1, len(train_loss_history) + 1)

fig, axs = plt.subplots(1, 2, figsize=(10, 3))
# Plot Training Loss over Epochs
axs[0].plot(epochs, train_loss_history)
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Training Loss')
# Plot Bounds over Epochs
axs[1].plot(epochs, bounds_history, label='Bounds', color='orange')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Bounds')

pf.fix_plot([axs])

plt.tight_layout()
plt.savefig(f'figs/{netid}/bounds.png', dpi=300)
plt.tight_layout();

# Distotions enveloping the data
bounds_history = [bound.item() if hasattr(bound, 'item') else bound for bound in model.bounds_history]
sample = simulator.sample(1)
ni = sample['ni']

fig, axs = plt.subplots(4, 5, figsize=(20, (4+1)*3), sharex=True)
axs = axs.flatten()
for i_b, b in enumerate(bounds_history):
    if i_b < len(axs):
        axs[i_b].set_title(r"$b$ = {:.2f}".format(b))
        for j in range(10):
            sample = simulator.sample(1)
            ni = sample['ni']
            # axs[i_b].plot(sample['mu'][0].cpu(), c='k', ls='--')
            epsilon_sim =  (2 * b * torch.rand(sample['xi'].shape, device= sample['xi'].device, dtype= sample['xi'].dtype) - b) * ni
            data =  sample['x0'] + epsilon_sim * ni
            axs[i_b].plot(data[0].cpu(), c='C0', alpha=0.4)

pf.fix_plot([axs])
plt.tight_layout()
plt.savefig(f'figs/{netid}/history.png', dpi=300)


In [ ]:


network_epsilon.cuda()
N_mc = 2e6

obs = simulator.sample(1)
ni = torch.eye(Nbins, dtype=obs['xi'].dtype)
variance = 1 / get_sigma_epsilon_inv2(ni)

batch_size = 2048*2
N_batch = int(N_mc / batch_size)

ts_bin_H0_epsilon = []
for _ in tqdm(range(N_batch)):
    mc_samples = simulator.sample(batch_size)
    ts_batch =  (network_epsilon.snr(mc_samples['x0'].cuda())**2).detach().cpu().numpy()
    ts_bin_H0_epsilon.append(ts_batch)
    
ts_bin_H0_epsilon = np.concatenate(ts_bin_H0_epsilon)


In [ ]:
def pvalue_grid_eps(dat):
    eps_t_mean = np.mean(ts_bin_H0_epsilon, axis=0)
    eps_t_ref = ts_bin_H0_epsilon - eps_t_mean
    counts = np.sum(eps_t_ref >= dat[:, np.newaxis, :], axis=1)
    return (counts + 1) / (len(eps_t_ref) + 1)

def pvalue_grid_BCE(dat):
    BCE_t_mean = np.mean(ts_bin_H0_BCE, axis=0)
    BCE_t_ref = ts_bin_H0_BCE - BCE_t_mean
    counts = np.sum(BCE_t_ref >= dat[:, np.newaxis, :], axis=1)
    return (counts + 1) / (len(BCE_t_ref) + 1)

def chop_middle(array, remove=5, linemode=True):
    if len(array)%2==0:
        mid_u = int(len(array)/4)
        mid_d = mid_u -1
        if not linemode:
            return np.concatenate([array[:mid_d-remove], array[mid_u+remove:]])
        else:
            return array[:mid_d-remove] , array[mid_u+remove:], array[mid_u]
    else:
        mid = len(array)//2
        if not linemode:
            return np.concatenate([array[:mid-remove], array[mid+remove:]])
        else:
            return array[:mid-remove], array[mid+remove:], array[mid]
        
def get_snr2(input:dict):
    target = input['x']
    snr2_nn = network_epsilon.snr(target.cuda()).detach().cpu().numpy()**2 
    return snr2_nn

def local_do_ticks(list_of_axes, dir = 'in'):
    for ax in list_of_axes:
        ax.minorticks_on()
        ax.tick_params(top=True,right=True, direction=dir, length=7, which='major')
        ax.tick_params(top=True,right=True, direction=dir, length=4, which='minor')

def local_fix_frame(ax):
    ax.tick_params(color='black', labelcolor='black')
    ax.spines[:].set_color('black')
    ax.spines[:].set_linewidth(1)
    return True

def local_fix_plot(a, tickdir ='in'):
    for axes in a:
        axes.grid(False)
        local_do_ticks([axes], tickdir)
        local_fix_frame(axes)
    return True

In [ ]:
###### SET UP GRID ######
positions = torch.arange(0, Nbins, 1).to(dtype=simulator.dtype)
amplitudes = torch.linspace(-3, 10, 80).to(dtype=simulator.dtype)

position_grid, amplitude_grid = torch.meshgrid(positions,amplitudes)
b = {'x':amplitude_grid.T}

###### DO BCE STATS ######
a = b['x']
s = get_snr2(b)
dat = [a,s]


fig, ax1 = pf.create_plot()
ax1.set_xlabel(r'$f$')
ax2 = fig.add_axes((1.05, 0,0.1,1))

# ax3 = fig.add_axes((0, 1.1,1,1))
# plt.setp(ax3.get_xticklabels(), visible=False)
# ax4 = fig.add_axes((1.05, 1.1,0.1,1))

axs = [ax1,ax2]


dat = [pvalue_grid_eps(s)]
lab =  [r'$\mathrm{log}_{10}($p$_{i, \mathrm{SNR}})$']

labcolour = "#000000"

obs = simulator.sample(100)


for q in range(1):
    mesh = axs[2*q].pcolormesh(position_grid.T, amplitude_grid.T, np.log10(dat[q]), cmap='magma', vmin=-8)
    fig.colorbar(mesh,cax=axs[2*q+1], shrink=0.8, label=lab[q])
    axs[2*q+1].set_ylim([-6.5,0])

    for j in range(2):
        axs[2*q].plot(chop_middle(positions)[j], chop_middle(obs['mu'][0])[j], color=labcolour, linewidth=3)
        for i in range(5):
            alp = .5+(i/8)
            axs[2*q].plot(chop_middle(positions)[j], chop_middle(obs['mu'][0]+quantiles[-i])[j], color=labcolour, alpha=alp)

    x = 25
    axs[2*q].text(x,obs['mu'][0][int(x)], r'$\mu$', color=labcolour, size=20)
    sigs = [r'$+3\sigma$',r'$+2\sigma$',r'$+\sigma$',r'$\bar{x}_0$']
    x2 = 25
    ff = torch.Tensor([0,0,0,-1])
    for i in range(4):
        axs[2*q].text(x2,(obs['mu'][0]+quantiles[-i])[int(x2)], sigs[i-1], color=labcolour, size=12, ha='center')  

    axs[2*q].set_ylabel(r'$\tilde{d}(f)$')

local_fix_plot(axs, tickdir='out')

plt.tight_layout()
plt.savefig(f'figs/{netid}/pmaps.png', dpi=700, bbox_inches = 'tight')


In [ ]:
torch.from_numpy(simulator._fd_theta_batched(100))